# FARLAB - The Robotability Score 
Developer: @mattwfranchi

In this notebook, we load, preprocess, and aggregate different types of street furniture to come up with a 'street clutter' metric, at the per-sidewalk level.

Project Members: Matt Franchi, Maria-Teresa Parreira, Frank Bu, Wendy Ju 

In [1]:
# class RobotabilityGraph that inherits from Graph class 
import os
import sys 
sys.path.append("/share/ju/urban-fingerprinting")

import osmnx as ox 
import geopandas as gpd 
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
# enable latex plotting 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from glob import glob 
from tqdm import tqdm 

from shapely import wkt, LineString 

import rasterio
from rasterio.enums import Resampling
from rasterio.plot import show 


from src.utils.logger import setup_logger 

logger = setup_logger('rs-street-furniture')
logger.setLevel("INFO")
logger.info("Modules initialized.")

WGS='EPSG:4326'
PROJ='EPSG:2263'

REGEN_SEGMENTIZATION=False
REGEN_TOPOLOGY=True

GEN_INSPECTION_PLOTS=True
INSPECTION_PLOTS="figures/inspection_plots"

os.makedirs(INSPECTION_PLOTS, exist_ok=True)


2024-08-29 11:42:53 - rs-street-furniture - INFO - Modules initialized.


## Loading and Preprocessing Data Features 

### Sidewalk Basemap (NYC)

In [2]:
segmentized = pd.read_csv("../data/sidewalks_nyc_segmentized.csv")
segmentized = gpd.GeoDataFrame(segmentized, geometry=segmentized['geometry'].apply(wkt.loads), crs=PROJ)
logger.info("Segmentized sidewalk basemap loaded.")


sidewalk_nyc = segmentized

logger.success("NYC sidewalk basemap loaded.")
logger.info(f"Distribution of sidewalk widths [ft]: \n{sidewalk_nyc['SHAPE_Width'].describe()}")

2024-08-29 11:43:15 - rs-street-furniture - INFO - Segmentized sidewalk basemap loaded.
2024-08-29 11:43:15 - rs-street-furniture - SUCCESS - NYC sidewalk basemap loaded.
2024-08-29 11:43:15 - rs-street-furniture - INFO - Distribution of sidewalk widths [ft]: 
count    2.551214e+06
mean     5.373539e+00
std      1.480773e+00
min      2.710948e-01
25%      4.458703e+00
50%      5.149090e+00
75%      6.177234e+00
max      4.021491e+01
Name: SHAPE_Width, dtype: float64


In [3]:
# the maximum distance to search for a nearby street segment. Since we segmentize by 50 feet, we can search within 25 feet
MAX_DISTANCE=25

CUTOFF= pd.to_datetime("2023-08-31")


In [4]:

# we buffer each point by 25 feet, creating a 50-diameter circle centered at the point. This captures nearby clutter. 
sidewalk_nyc['geometry'] = sidewalk_nyc['geometry'].buffer(MAX_DISTANCE)

### Bus Stop Shelters 

In [5]:
# read bus stop shelters 
bus_stop_shelters = gpd.read_file("../data/street_furniture/bus_stop_shelters_nyc.csv")
bus_stop_shelters = gpd.GeoDataFrame(bus_stop_shelters, geometry=wkt.loads(bus_stop_shelters['the_geom']), crs=WGS).to_crs(PROJ)

# Bus stop installation date is not present, so filtering is out-of-scoped.

### Trash Cans / Waste Baskets 

In [6]:
# load trash cans 
trash_cans = pd.read_csv("../data/street_furniture/dsny_litter_baskets_nyc.csv")
trash_cans = gpd.GeoDataFrame(trash_cans, geometry=wkt.loads(trash_cans['point']), crs=WGS).to_crs(PROJ)

# trash can installation date is not present, so filtering is out-of-scope

### LinkNYC Kiosks 

In [7]:
# load linknyc
linknyc = pd.read_csv("../data/street_furniture/LinkNYC_Kiosk_Locations_20240816.csv")
linknyc = gpd.GeoDataFrame(linknyc, geometry=gpd.points_from_xy(linknyc['Longitude'], linknyc['Latitude']), crs='EPSG:4326').to_crs(PROJ)

linknyc['Installation Complete'] = pd.to_datetime(linknyc['Installation Complete'])
linknyc = linknyc[linknyc['Installation Complete'] <= CUTOFF]
linknyc['Installation Complete'].describe()

count                             2137
mean     2017-12-17 15:39:20.131024896
min                1971-12-01 00:00:00
25%                2016-11-12 00:00:00
50%                2017-07-19 00:00:00
75%                2018-02-17 00:00:00
max                2023-11-30 00:00:00
Name: Installation Complete, dtype: object

### Bicycle Parking Shelters 

In [8]:
# load bicycle parking shelters 
bicycle_parking_shelters = pd.read_csv("../data/street_furniture/bicycle_parking_shelters_nyc.csv")
bicycle_parking_shelters = gpd.GeoDataFrame(bicycle_parking_shelters, geometry=wkt.loads(bicycle_parking_shelters['the_geom']), crs=WGS).to_crs(PROJ)
bicycle_parking_shelters['Build_date'] = pd.to_datetime(bicycle_parking_shelters['Build_date'])
bicycle_parking_shelters = bicycle_parking_shelters[bicycle_parking_shelters['Build_date'] <= CUTOFF]
bicycle_parking_shelters['Build_date'].describe()

/tmp/ipykernel_4166785/4045691724.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bicycle_parking_shelters['Build_date'] = pd.to_datetime(bicycle_parking_shelters['Build_date'])


count                               17
mean     2008-07-26 18:21:10.588235264
min                2007-12-17 00:00:00
25%                2008-07-01 00:00:00
50%                2008-09-12 00:00:00
75%                2008-10-13 00:00:00
max                2008-12-17 00:00:00
Name: Build_date, dtype: object

### Bicycle Racks 

In [9]:
# load bicycle racks 
bicycle_racks = gpd.read_file("../data/street_furniture/bicycle_racks_nyc/Bycycle_Parking_2023_07/Bicycle_Parking_2023_07.shp").to_crs(PROJ)
bicycle_racks['Date_Inst'] = pd.to_datetime(bicycle_racks['Date_Inst'])
bicycle_racks = bicycle_racks[bicycle_racks['Date_Inst'] <= CUTOFF]
bicycle_racks['Date_Inst'].describe()

count                            31585
mean     2011-06-02 09:53:44.100047616
min                1900-01-01 00:00:00
25%                2010-06-14 00:00:00
50%                2013-05-31 00:00:00
75%                2017-03-02 00:00:00
max                2023-05-02 00:00:00
Name: Date_Inst, dtype: object

### CityBench 

In [10]:
# load citybench
citybench = pd.read_csv("../data/street_furniture/citybench_nyc.csv")
citybench = gpd.GeoDataFrame(citybench, geometry=gpd.points_from_xy(citybench['Longitude'], citybench['Latitude']), crs='EPSG:4326').to_crs(PROJ)
citybench['Installati'] = pd.to_datetime(citybench['Installati'])
citybench = citybench[citybench['Installati'] <= CUTOFF]
citybench['Installati'].describe()

count                             2141
mean     2015-04-09 03:36:34.301728256
min                2012-04-01 00:00:00
25%                2013-07-10 00:00:00
50%                2014-11-18 00:00:00
75%                2017-02-04 00:00:00
max                2019-09-30 00:00:00
Name: Installati, dtype: object

### Street Trees 

In [11]:
# load trees 
trees = pd.read_csv("../data/street_furniture/forestry_tree_points_nyc.csv", engine='pyarrow')
trees = gpd.GeoDataFrame(trees, geometry=gpd.points_from_xy(trees['longitude'], trees['latitude']), crs='EPSG:4326').to_crs(PROJ)
trees['created_at'] = pd.to_datetime(trees['created_at'])
trees = trees[trees['created_at'] <= CUTOFF]
trees['created_at'].describe()

count                           683788
mean     2015-12-06 07:18:59.574254592
min                2015-05-19 00:00:00
25%                2015-08-29 00:00:00
50%                2015-10-23 00:00:00
75%                2016-02-06 00:00:00
max                2016-10-05 00:00:00
Name: created_at, dtype: object

### News Stands 

In [12]:
# load newsstands 
newsstands = pd.read_csv("../data/street_furniture/newsstands_nyc.csv", engine='pyarrow')
newsstands = gpd.GeoDataFrame(newsstands, geometry=wkt.loads(newsstands['the_geom']), crs='EPSG:4326').to_crs(PROJ)
newsstands['Built_Date'] = pd.to_datetime(newsstands['Built_Date'])
newsstands = newsstands[newsstands['Built_Date'] <= CUTOFF]
newsstands['Built_Date'].describe() 

count                              357
mean     2011-09-04 23:31:45.882352896
min                2007-09-03 00:00:00
25%                2008-07-30 00:00:00
50%                2010-11-08 00:00:00
75%                2013-06-04 00:00:00
max                2021-12-22 00:00:00
Name: Built_Date, dtype: object

### Parking Meters 

In [13]:
# load parking meters 
parking_meters = pd.read_csv("../data/street_furniture/parking_meters_nyc.csv")
parking_meters = gpd.GeoDataFrame(parking_meters, geometry=wkt.loads(parking_meters['Location']), crs='EPSG:4326').to_crs(PROJ)

# parking meter installation date is not present, so filtering is out-of-scope

### Fire Hydrants 

In [14]:
# load hydrants 
hydrants = pd.read_csv("../data/street_furniture/fire_hydrants_nyc.csv")
hydrants = gpd.GeoDataFrame(hydrants, geometry=wkt.loads(hydrants['the_geom']), crs='EPSG:4326').to_crs(PROJ)

# hydrant installation date is not present, so filtering is out-of-scope

### Street Signs 

In [15]:
# load street signs 
street_signs = pd.read_csv("../data/street_furniture/Street_Sign_Work_Orders_20240816.csv", engine='pyarrow')

# only keep 'Current' record type 
street_signs = street_signs[street_signs['record_type'] == 'Current']
street_signs['order_completed_on_date'] = pd.to_datetime(street_signs['order_completed_on_date'])
street_signs = street_signs[street_signs['order_completed_on_date'] <= CUTOFF]
street_signs = gpd.GeoDataFrame(street_signs, geometry=gpd.points_from_xy(street_signs['sign_x_coord'], street_signs['sign_y_coord']), crs=PROJ)
street_signs['order_completed_on_date'].describe()

count                          1071145
mean     2018-10-01 02:10:48.026551040
min                1953-08-07 00:00:00
25%                2017-08-23 00:00:00
50%                2020-01-07 00:00:00
75%                2021-07-15 00:00:00
max                2023-12-02 00:00:00
Name: order_completed_on_date, dtype: object

### Bollards 

In [16]:
# load bollards 
bollards = pd.read_csv("../data/street_furniture/Traffic_Bollards_Tracking_and_Installations_20240816.csv", engine='pyarrow')

bollards['Date'] = pd.to_datetime(bollards['Date'])
bollards = bollards[bollards['Date'] <= CUTOFF]
bollards['Date'].describe()

# we choose not to process bollards, as locations need to be geocoded. Latitude/Longitude is not present in the dataset.

count                            54665
mean     2016-05-17 19:10:59.939632384
min                2005-01-10 00:00:00
25%                2012-09-10 00:00:00
50%                2017-07-12 00:00:00
75%                2020-06-25 00:00:00
max                2023-08-31 00:00:00
Name: Date, dtype: object

### In-Service Fire Alarm Call Boxes 

In [17]:
alarm_call_boxes = pd.read_csv("../data/street_furniture/In-Service_Alarm_Box_Locations_20240816.csv")
alarm_call_boxes = gpd.GeoDataFrame(alarm_call_boxes, geometry=wkt.loads(alarm_call_boxes['Location Point']), crs='EPSG:4326').to_crs(PROJ)

### Scaffolding 

In [19]:
# read DoB active scaffolding permits 
scaffolding_permits = pd.read_csv("../data/dob_active_sheds.csv", engine='pyarrow')
scaffolding_permits = gpd.GeoDataFrame(scaffolding_permits, geometry=gpd.points_from_xy(scaffolding_permits['Longitude Point'], scaffolding_permits['Latitude Point']), crs='EPSG:4326')
scaffolding_permits = scaffolding_permits.to_crs(PROJ)

In [20]:
scaffolding_permits['First Permit Date']  = pd.to_datetime(scaffolding_permits['First Permit Date'])

scaffolding_permits = scaffolding_permits[scaffolding_permits['First Permit Date'] <= CUTOFF]
scaffolding_permits['First Permit Date'].describe()

count                             7771
mean     2022-07-19 11:03:34.489769728
min                2006-04-27 00:00:00
25%                2022-02-10 00:00:00
50%                2023-02-08 00:00:00
75%                2023-07-17 00:00:00
max                2023-12-01 00:00:00
Name: First Permit Date, dtype: object

## Spatial Joining of Street Furnitures to Sidewalk Graph 

In [21]:
# sjoin nearest bus stops and trash cans to sidewalk
len_before = len(sidewalk_nyc)
bus_stop_shelters = gpd.sjoin(sidewalk_nyc, bus_stop_shelters, )
logger.info(f"Missing {len(bus_stop_shelters[bus_stop_shelters['index_right'].isna()])} bus stop shelters.")

2024-08-29 11:45:05 - rs-street-furniture - INFO - Missing 0 bus stop shelters.


In [22]:
# sjoin nearest trash cans to sidewalk
len_before = len(trash_cans)
trash_cans = gpd.sjoin(sidewalk_nyc, trash_cans, )
logger.info(f"Removed {len_before - len(trash_cans)} trash cans that are not on sidewalks.")

2024-08-29 11:45:06 - rs-street-furniture - INFO - Removed 1514 trash cans that are not on sidewalks.


In [23]:
# sjoin nearest linknyc to sidewalk
len_before = len(linknyc)
linknyc = gpd.sjoin(sidewalk_nyc, linknyc, )
logger.info(f"LinkNYC: {len_before} -> {len(linknyc)}")

2024-08-29 11:45:07 - rs-street-furniture - INFO - LinkNYC: 2137 -> 4529


In [24]:
# sjoin nearest citybench 
len_before = len(citybench)
citybench = gpd.sjoin(sidewalk_nyc, citybench, )
logger.info(f"Citybench: {len_before} -> {len(citybench)}")

2024-08-29 11:45:08 - rs-street-furniture - INFO - Citybench: 2141 -> 1336


In [25]:
# sjoint nearest bicycle parking shelters to sidewalk
len_before = len(bicycle_parking_shelters)
bicycle_parking_shelters = gpd.sjoin(sidewalk_nyc, bicycle_parking_shelters, )
logger.info(f"Bicycle Parking Shelters: {len_before} -> {len(bicycle_parking_shelters)}")

2024-08-29 11:45:08 - rs-street-furniture - INFO - Bicycle Parking Shelters: 17 -> 35


In [26]:

# sjoin nearest bicycle racks to sidewalk
len_before = len(bicycle_racks)
bicycle_racks = gpd.sjoin(sidewalk_nyc, bicycle_racks, )
logger.info(f"Bicycle Racks: {len_before} -> {len(bicycle_racks)}")

2024-08-29 11:45:10 - rs-street-furniture - INFO - Bicycle Racks: 31585 -> 57730


In [27]:
# sjoin nearest trees to sidewalk
len_before = len(trees)
trees = gpd.sjoin(sidewalk_nyc, trees, )
logger.info(f"Trees: {len_before} -> {len(trees)}")

2024-08-29 11:45:17 - rs-street-furniture - INFO - Trees: 683788 -> 1369979


In [28]:
# sjoin nearest newsstands to sidewalk
len_before = len(newsstands)
newsstands = gpd.sjoin(sidewalk_nyc, newsstands, )
logger.info(f"Newsstands: {len_before} -> {len(newsstands)}")

2024-08-29 11:45:18 - rs-street-furniture - INFO - Newsstands: 357 -> 537


In [29]:
# sjoin nearest parking meters to sidewalk
len_before = len(parking_meters)
parking_meters = gpd.sjoin(sidewalk_nyc, parking_meters, )
logger.info(f"Parking Meters: {len_before} -> {len(parking_meters)}")

2024-08-29 11:45:19 - rs-street-furniture - INFO - Parking Meters: 13336 -> 25913


In [30]:
# sjoin nearest hydrants to sidewalk
len_before = len(hydrants)
hydrants = gpd.sjoin(sidewalk_nyc, hydrants, )
logger.info(f"Hydrants: {len_before} -> {len(hydrants)}")

2024-08-29 11:45:20 - rs-street-furniture - INFO - Hydrants: 109410 -> 215766


In [31]:
# sjoin nearest street signs to sidewalk
len_before = len(street_signs)
street_signs = gpd.sjoin(sidewalk_nyc, street_signs, )
logger.info(f"Street Signs: {len_before} -> {len(street_signs)}")

2024-08-29 11:45:26 - rs-street-furniture - INFO - Street Signs: 1071145 -> 845353


In [32]:
# sjoin nearest bollards to sidewalk
#len_before = len(bollards)
#bollards = gpd.sjoin(sidewalk_nyc, bollards )
#logger.info(f"Bollards: {len_before} -> {len(bollards)}")


In [33]:
# sjoin nearest alarm call boxes to sidewalk
len_before = len(alarm_call_boxes)
alarm_call_boxes = gpd.sjoin(sidewalk_nyc, alarm_call_boxes, )
logger.info(f"Alarm Call Boxes: {len_before} -> {len(alarm_call_boxes)}")

2024-08-29 11:45:27 - rs-street-furniture - INFO - Alarm Call Boxes: 13008 -> 9746


In [35]:
BUFFER=100 
# buffer scaffolding_permits points, then sjoin to sidewalks
scaffolding_permits.geometry = scaffolding_permits.geometry.buffer(BUFFER)
scaffolding_permits = gpd.sjoin(sidewalk_nyc, scaffolding_permits, predicate='intersects')

/share/ju/conda_virtualenvs/urbanekg/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [36]:

# now, get number of bus stops, trash cans, linknyc, citybench, bicycle parking shelters, and bicycle racks per sidewalk
bus_stop_counts = bus_stop_shelters.groupby('point_index').size().reset_index(name='bus_stop_count').fillna(0)
trash_can_counts = trash_cans.groupby('point_index').size().reset_index(name='trash_can_count').fillna(0)
linknyc_counts = linknyc.groupby('point_index').size().reset_index(name='linknyc_count').fillna(0)
citybench_counts = citybench.groupby('point_index').size().reset_index(name='citybench_count').fillna(0)
bicycle_parking_shelter_counts = bicycle_parking_shelters.groupby('point_index').size().reset_index(name='bicycle_parking_shelter_count').fillna(0)
bicycle_rack_counts = bicycle_racks.groupby('point_index').size().reset_index(name='bicycle_rack_count').fillna(0)
tree_counts = trees.groupby('point_index').size().reset_index(name='tree_count').fillna(0)
newsstand_counts = newsstands.groupby('point_index').size().reset_index(name='newsstand_count').fillna(0)
parking_meter_counts = parking_meters.groupby('point_index').size().reset_index(name='parking_meter_count').fillna(0)
hydrant_counts = hydrants.groupby('point_index').size().reset_index(name='hydrant_count').fillna(0)
street_sign_counts = street_signs.groupby('point_index').size().reset_index(name='street_sign_count').fillna(0)
#bollard_counts = bollards.groupby('point_index').size().reset_index(name='bollard_count').fillna(0)
alarm_call_box_counts = alarm_call_boxes.groupby('point_index').size().reset_index(name='alarm_call_box_count').fillna(0)

In [37]:
# merge scaffolding in 
scaffolding_counts = scaffolding_permits.groupby('point_index').size().reset_index(name='scaffolding_permit_count').fillna(0)

In [38]:

# merge counts to sidewalk_nyc
sidewalk_nyc = sidewalk_nyc.merge(bus_stop_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(trash_can_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(linknyc_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(citybench_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(bicycle_parking_shelter_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(bicycle_rack_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(tree_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(newsstand_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(parking_meter_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(hydrant_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(street_sign_counts, on='point_index', how='left')
#sidewalk_nyc = sidewalk_nyc.merge(bollard_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(alarm_call_box_counts, on='point_index', how='left')

In [40]:
# merge scaffolding in 
sidewalk_nyc = sidewalk_nyc.merge(scaffolding_counts, on='point_index', how='left')

In [41]:

sidewalk_nyc = sidewalk_nyc.fillna(0)

In [42]:
sidewalk_nyc.describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

,Unnamed: 0,SOURCE_ID,SHAPE_Leng,SHAPE_Area,SHAPE_Width,point_index,bus_stop_count,trash_can_count,linknyc_count,citybench_count,bicycle_parking_shelter_count,bicycle_rack_count,tree_count,newsstand_count,parking_meter_count,hydrant_count,street_sign_count,alarm_call_box_count,scaffolding_permit_count
count,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06,2.551214e+06
mean,1.275606e+06,1.331676e+10,5.353440e+03,3.053169e+04,5.373539e+00,1.275606e+06,2.705771e-03,9.080775e-03,1.775233e-03,5.236723e-04,1.371896e-05,2.262844e-02,5.369910e-01,2.104880e-04,1.015713e-02,8.457385e-02,3.313532e-01,3.820142e-03,2.144434e-01
std,7.364722e+05,5.368509e+09,1.268543e+04,8.059542e+04,1.480773e+00,7.364722e+05,5.272552e-02,9.501635e-02,4.213334e-02,3.807908e-02,3.703887e-03,2.311208e-01,7.353800e-01,1.458752e-02,1.003554e-01,2.793039e-01,1.178561e+00,6.571934e-02,7.596916e-01
min,0.000000e+00,0.000000e+00,1.388815e+01,1.018359e+01,2.710948e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,2.551213e+04,1.380000e+09,2.233946e+02,7.566396e+02,2.257815e+00,2.551213e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2.5%,6.378033e+04,1.380001e+09,4.735645e+02,1.814837e+03,2.724753e+00,6.378033e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5%,1.275607e+05,3.380000e+09,8.330682e+02,3.479572e+03,3.319015e+00,1.275607e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
10%,2.551213e+05,5.380001e+09,1.453053e+03,6.537684e+03,3.818261e+00,2.551213e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.378032e+05,9.380001e+09,2.449058e+03,1.202621e+04,4.458703e+00,6.378032e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.275606e+06,1.438000e+10,3.306964e+03,1.699473e+04,5.149090e+00,1.275606e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [43]:
# naive weights based on predicted area of different clutters 
weights = { 
    'bus_stop_count': 2,
    'trash_can_count': 0.5, 
    'linknyc_count': 2, 
    'citybench_count': 1.5,
    'bicycle_parking_shelter_count': 2,
    'bicycle_rack_count': 1.5,
    'tree_count': .15,
    'newsstand_count': 3, 
    'parking_meter_count': .15,
    'scaffolding_permit_count': 2,
    'hydrant_count': 0.25,
    'street_sign_count': 0.05,
    #'bollard_count': 0.05,
    'alarm_call_box_count': .15
}

In [44]:

# create a 'clutter' metric that is the sum of all street clutter features
sidewalk_nyc['clutter'] = 0
for feature, weight in weights.items():
    sidewalk_nyc['clutter'] += sidewalk_nyc[feature] * weight

sidewalk_nyc['clutter'].describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

count    2.551214e+06
mean     5.981326e-01
std      1.606934e+00
min      0.000000e+00
1%       0.000000e+00
2.5%     0.000000e+00
5%       0.000000e+00
10%      0.000000e+00
25%      0.000000e+00
50%      1.500000e-01
75%      3.000000e-01
90%      2.050000e+00
95%      3.650000e+00
97.5%    4.500000e+00
99%      8.000000e+00
max      4.450000e+01
Name: clutter, dtype: float64

In [45]:
# Now, weighted clutter by sidewalk width 
sidewalk_nyc['clutter'] = sidewalk_nyc['clutter'] / sidewalk_nyc['SHAPE_Width']

In [46]:
# clamp distribution to 1st and 99th percentile
sidewalk_nyc['clutter'] = sidewalk_nyc['clutter'].clip(lower=sidewalk_nyc['clutter'].quantile(0.01), upper=sidewalk_nyc['clutter'].quantile(0.99))

In [47]:
# final describe 
sidewalk_nyc['clutter'].describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

count    2.551214e+06
mean     9.642424e-02
std      2.168543e-01
min      0.000000e+00
1%       0.000000e+00
2.5%     0.000000e+00
5%       0.000000e+00
10%      0.000000e+00
25%      0.000000e+00
50%      2.539171e-02
75%      5.867995e-02
90%      3.366669e-01
95%      5.479976e-01
97.5%    8.310632e-01
99%      1.267572e+00
max      1.267572e+00
Name: clutter, dtype: float64

In [48]:
# write street furniture density to csv 
os.makedirs("../data/processed", exist_ok=True)
sidewalk_nyc.to_csv("../data/processed/street_furniture_density.csv", index=False)

In [49]:
# map sidewalk and color by clutter 
fig, ax = plt.subplots(figsize=(20, 20))
sidewalk_nyc.plot(column='clutter', ax=ax, legend=True, cmap='cividis', markersize=0.25, legend_kwds={'label': "Weighted Street Clutter", 'orientation': 'horizontal', 'shrink': 0.5, 'pad': 0.01})
ax.set_axis_off()

plt.savefig("figures/street_furniture_density.png", dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()